<a href="https://colab.research.google.com/github/LeomaxFilho/Operational-Research/blob/main/Pratica_Destinacao_Produtos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Aula prática: Destinação de Produtos à Fabricas
<sup>Exemplo do capítulo 8.3 do livro `Introdução à Pesquisa Operacional` de `Hillier e Lieberman`.</sup>

## Exercício 1

### Descrição do problema
A Cia. Produtos Melhores decidiu iniciar a produção de quatro produtos novos usando três fábricas que, no momento, têm excesso de capacidade produtiva. Os produtos requerem um esforço de produção comparável por unidade, de modo que a capacidade produtiva disponível das fábricas seja medida pelo número de unidades de qualquer produto que possa ser produzido diariamente, conforme dado na tabela 1. A tabela 2 fornece a taxa de produção diária necessária para atender às vendas projetadas. Cada fábrica é capaz de produzir qualquer um desses produtos, exceto a Fábrica 2, que não pode fabricar o produto 3. Os custos variáveis por unidade de cada produto diferem de fábrica para fábrica, conforme a tabela 3.

Tabela 1: Capacidade de produção disponível, por unidade do produto

| | Fábrica 1 | Fábrica 2 | Fábrica 3 |
|:---|:---:|:---:|:---:|
| Capacidade disponível | 75 | 75 | 45 |

Tabela 2: Demanda de produção diária

| | Produto 1 | Produto 2 | Produto 3 | Produto 4 |
|:---|:---:|:---:|:---:|:---:|
| Demanda | 20 | 30 | 30 | 40 |

Tabela 3: Custo unitário por produto

| | Produto 1 | Produto 2 | Produto 3 | Produto 4 |
|:---|:---:|:---:|:---:|:---:|
| Fábrica 1 | 41 | 27 | 28 | 24 |
| Fábrica 2 | 40 | 29 | - | 23 |
| Fábrica 3 | 37 | 30 | 27 | 21 |

A gerência precisa tomar uma decisão sobre como dividir a fabricação dos produtos entre as fábricas, e decidiu permitir a divisão da produção de um mesmo produto em mais de uma fábrica.

Escreva um modelo para determinar quais fábricas produzirão quais produtos e a que custo total.

### Resolução

In [2]:
# instalação e importação do pacote mip
!pip install mip
from mip import *

# funcões usadas posteriormente:

# resolve o modelo e mostra os valores das variáveis
def solve(model):
  status = model.optimize()

  print("Status = ", status)
  print(f"Solution value  = {model.objective_value:.2f}\n")

  print("Solution:")
  for v in model.vars:
      print(f"{v.name} = {v.x:.2f}")


# salva modelo em arquivo lp, e mostra o conteúdo
def save(model, filename):
  model.write(filename) # salva modelo em arquivo
  with open(filename, "r") as f: # lê e exibe conteúdo do arquivo
    print(f.read())

In [13]:
F = [75, 75, 45]
P = [20, 30, 30, 40]
CS = [[41, 27, 28, 24],[40, 29, 0, 23], [37, 30, 27, 21]]

In [26]:
model = Model(name='Destinacao de Produtos a Fabricas', sense=MINIMIZE, solver_name=CBC)

x = {i: {j: model.add_var(name = f"x_{i}_{j}") for j in range(len(P))} for i in range(len(F))}


model.objective += xsum(CS[i][j] * x[i][j] for j in range(len(P)) for i in range(len(F)))

model += x[1][2] == 0

for i in range(len(F)):
  model += xsum(x[i][j] for j in range(len(P))) <= F[i]

for j in range(len(P)):
  model += xsum(x[i][j] for i in range(len(F))) >= P[j]

In [27]:
save(model, "model.lp")

\Problem name: Destinacao de Produtos a Fabricas

Minimize
OBJROW: 41 x_0_0 + 27 x_0_1 + 28 x_0_2 + 24 x_0_3 + 40 x_1_0 + 29 x_1_1 + 23 x_1_3 + 37 x_2_0 + 30 x_2_1 + 27 x_2_2
 + 21 x_2_3
Subject To
constr(0):  x_1_2 = 0
constr(1):  x_0_0 + x_0_1 + x_0_2 + x_0_3 <= 75
constr(2):  x_1_0 + x_1_1 + x_1_2 + x_1_3 <= 75
constr(3):  x_2_0 + x_2_1 + x_2_2 + x_2_3 <= 45
constr(4):  x_0_0 + x_1_0 + x_2_0 >= 20
constr(5):  x_0_1 + x_1_1 + x_2_1 >= 30
constr(6):  x_0_2 + x_1_2 + x_2_2 >= 30
constr(7):  x_0_3 + x_1_3 + x_2_3 >= 40
Bounds
End



In [28]:
solve(model)

Status =  OptimizationStatus.OPTIMAL
Solution value  = 3260.00

Solution:
x_0_0 = 0.00
x_0_1 = 30.00
x_0_2 = 30.00
x_0_3 = 0.00
x_1_0 = 0.00
x_1_1 = 0.00
x_1_2 = 0.00
x_1_3 = 15.00
x_2_0 = 20.00
x_2_1 = 0.00
x_2_2 = 0.00
x_2_3 = 25.00


## Exercício 2

Refaça o modelo anterior, dessa vez impedindo a divisão de produtos entre fábricas. Ou seja, cada produto deve ser integralmente produzido em uma única fábrica, para eliminar custos ocultos associados à divisão da produção. A cada fábrica deve ser destinado pelo menos um produto.

### Resolução

In [48]:
F = [75, 75, 45]
P = [20, 30, 30, 40]
CS = [[41, 27, 28, 24],[40, 29, 0, 23], [37, 30, 27, 21]]

In [54]:
model = Model(name='Destinacao de Produtos a Fabricas Impedindo Que Seja Dividido', sense=MINIMIZE, solver_name=CBC)

x = {i: {j: model.add_var(name = f"x_{i}_{j}", var_type=BINARY) for j in range(len(P))} for i in range(len(F))}

model.objective += xsum(CS[i][j] * P[j] * x[i][j] for j in range(len(P)) for i in range(len(F)))

for i in range(len(F)):
  model += xsum(x[i][j] for j in range(len(P)) ) >= 1
  model += xsum(P[j]*x[i][j] for j in range(len(P)) ) <= F[i]

for j in range(len(P)):
  model += xsum(P[j] * x[i][j] for i in range(len(F)) ) == P[j]

model += x[1][2] == 0

save(model, "model2.lp")

\Problem name: Destinacao de Produtos a Fabricas Impedindo Que Seja Dividido

Minimize
OBJROW: 820 x_0_0 + 810 x_0_1 + 840 x_0_2 + 960 x_0_3 + 800 x_1_0 + 870 x_1_1 + 920 x_1_3 + 740 x_2_0 + 900 x_2_1 + 810 x_2_2
 + 840 x_2_3
Subject To
constr(0):  x_0_0 + x_0_1 + x_0_2 + x_0_3 >= 1
constr(1):  20 x_0_0 + 30 x_0_1 + 30 x_0_2 + 40 x_0_3 <= 75
constr(2):  x_1_0 + x_1_1 + x_1_2 + x_1_3 >= 1
constr(3):  20 x_1_0 + 30 x_1_1 + 30 x_1_2 + 40 x_1_3 <= 75
constr(4):  x_2_0 + x_2_1 + x_2_2 + x_2_3 >= 1
constr(5):  20 x_2_0 + 30 x_2_1 + 30 x_2_2 + 40 x_2_3 <= 45
constr(6):  20 x_0_0 + 20 x_1_0 + 20 x_2_0 = 20
constr(7):  30 x_0_1 + 30 x_1_1 + 30 x_2_1 = 30
constr(8):  30 x_0_2 + 30 x_1_2 + 30 x_2_2 = 30
constr(9):  40 x_0_3 + 40 x_1_3 + 40 x_2_3 = 40
constr(10):  x_1_2 = 0
Bounds
 0 <= x_0_0 <= 1
 0 <= x_0_1 <= 1
 0 <= x_0_2 <= 1
 0 <= x_0_3 <= 1
 0 <= x_1_0 <= 1
 0 <= x_1_1 <= 1
 0 <= x_1_2 <= 1
 0 <= x_1_3 <= 1
 0 <= x_2_0 <= 1
 0 <= x_2_1 <= 1
 0 <= x_2_2 <= 1
 0 <= x_2_3 <= 1
Integers
x_0_0 x

In [55]:
solve(model)

Status =  OptimizationStatus.OPTIMAL
Solution value  = 3290.00

Solution:
x_0_0 = 0.00
x_0_1 = 1.00
x_0_2 = 1.00
x_0_3 = 0.00
x_1_0 = 1.00
x_1_1 = 0.00
x_1_2 = 0.00
x_1_3 = 0.00
x_2_0 = 0.00
x_2_1 = 0.00
x_2_2 = 0.00
x_2_3 = 1.00
